In [48]:
# RAG with gpt4-o
    

In [49]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

# Get a Data Loader

In [50]:
from langchain_community.document_loaders import WebBaseLoader

In [51]:
loader = WebBaseLoader("https://www.investopedia.com/how-nvidia-makes-money-4799532#:~:text=Nvidia%20(NVDA)%20introduced%20graphics%20processing,vehicles%2C%20robotics%2C%20and%20more.")
data = loader.load()
data

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\nHow Nvidia Makes Money\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n\n\n\n\n\n\n\nBy clicking “Accept All Cookies”, you agree to the storing of cookies on your device to enhance site navigation, analyze site usage, and assist in our marketing efforts. Cookies Settings Accept All Cookies\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n Investing\n\n\n\n Stocks\n \n\n Cryptocurrency\n \n\n Bonds\n \n\n ETFs\n \n\n Options and Derivatives\n \n\n Commodities\n \n\n Trading\n \n\n Automated Investing\n \n\n Brokers\n \n\n Fundamental Analysis\n \n\n Markets\n \n\n View All\n \n\n\n\n Simulator\n\n\n\n Login / Portfolio\n \n\n Trade\n \n\n Research\n \n\n My Games\n \n\n Leaderboard\n \n\n\n\n Banking\n\n\n\n Savings Acc

# 2. Convert data to Vector Database

In [52]:
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [54]:
documents

[Document(page_content="How Nvidia Makes Money\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n\n\n\n\n\n\n\nBy clicking “Accept All Cookies”, you agree to the storing of cookies on your device to enhance site navigation, analyze site usage, and assist in our marketing efforts. Cookies Settings Accept All Cookies\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n Investing\n\n\n\n Stocks\n \n\n Cryptocurrency\n \n\n Bonds\n \n\n ETFs\n \n\n Options and Derivatives\n \n\n Commodities\n \n\n Trading\n \n\n Automated Investing\n \n\n Brokers\n \n\n Fundamental Analysis\n \n\n Markets\n \n\n View All\n \n\n\n\n Simulator\n\n\n\n Login / Portfolio\n \n\n Trade\n \n\n Research\n \n\n My Games\n \n\n Leaderboard\n \n\n\n\n Banking\n\n\n\n Savings Accounts\n \n\n Certifica

In [ ]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

# 3. Make a RAG pipeline

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [ ]:
llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [ ]:
question = "What should I do If I have Nvidia stock sell or hold"
result = qa_chain({"query": question })
result

{'query': 'What should I do If I have Nvidia stock sell or hold',
 'result': "Considering that Nvidia's earnings are expected to be in focus next week and the stock is currently at record levels, it might be prudent to hold your Nvidia stock until the earnings report is released. This could provide more clarity on the company's performance and future prospects. Always consider consulting with a financial advisor for personalized advice."}

In [ ]:
result["result"]

"It is suggested to keep an eye on Nvidia's earnings next week, as the stock is currently trying to sustain record levels. This upcoming financial report could provide more clarity on whether to sell or hold."

In [ ]:
question = "Who are the competitiors of Nvidia"
result = qa_chain({"query": question })
result

{'query': 'Who are the competitiors of Nvidia',
 'result': "Nvidia's competitors include Advanced Micro Devices (AMD), Intel, and other companies in the semiconductor and graphics processing industries."}